In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv
/kaggle/input/co2-emission-by-vehicles/Data Description.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
df=pd.read_csv("../input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv")
df.shape
df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [6]:
col = ['Engine Size(L)', 'Cylinders', 'Fuel Consumption Comb (L/100 km)', 'CO2 Emissions(g/km)']
df2 = df[col]
df2.shape
df2.head()

,Engine Size(L),Cylinders,Fuel Consumption Comb (L/100 km),CO2 Emissions(g/km)
0,2.0,4,8.5,196
1,2.4,4,9.6,221
2,1.5,4,5.9,136
3,3.5,6,11.1,255
4,3.5,6,10.6,244


In [8]:
df2.rename(columns = {"Engine Size(L)":"EngineSize", 'Cylinders':"Cylinders",
                      'Fuel Consumption Comb (L/100 km)':"FuelConsumption",
                      'CO2 Emissions(g/km)':"CO2Emissions"  }, 
          inplace = True
          )
df2.columns

Index(['EngineSize', 'Cylinders', 'FuelConsumption', 'CO2Emissions'], dtype='object')

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   EngineSize       7385 non-null   float64
 1   Cylinders        7385 non-null   int64  
 2   FuelConsumption  7385 non-null   float64
 3   CO2Emissions     7385 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 230.9 KB


In [10]:
df2.describe()

,EngineSize,Cylinders,FuelConsumption,CO2Emissions
count,7385.000000,7385.000000,7385.000000,7385.000000
mean,3.160068,5.615030,10.975071,250.584699
std,1.354170,1.828307,2.892506,58.512679
min,0.900000,3.000000,4.100000,96.000000
25%,2.000000,4.000000,8.900000,208.000000
50%,3.000000,6.000000,10.600000,246.000000
75%,3.700000,6.000000,12.600000,288.000000
max,8.400000,16.000000,26.100000,522.000000


In [14]:
X= df2.drop('CO2Emissions', axis=1)
y= df2['CO2Emissions']

In [16]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3)

In [17]:
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(5169, 3) (2216, 3) (5169,) (2216,)


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
model = LinearRegression()
model = model.fit(Xtrain, ytrain)


ypred = model.predict(Xtest)

model_score = r2_score(ytest, ypred)
model_rmse = mean_squared_error(ytest, ypred, squared = False)
model_acc = model.score(Xtest, ytest)
print("Predicted CO2: ", ypred[:4])
print("R2 Score : ", model_score)
print("RMSE : ", model_rmse)

Predicted CO2:  [300.50828388 270.05869309 391.73782655 222.76104314]
R2 Score :  0.8807917352115271
RMSE :  20.19631779746146


In [22]:
import pickle

pickle.dump(model, open('model.pkl','wb'))

In [26]:
#Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
print(model.predict([[3.5, 6, 10.6]]))

[249.69973463]


In [31]:
df2.to_csv('FuelConsumption.csv')

## Import necessary libraries, initialize the flask app, and load our ML model:
* We will initialize our app and then load the “model.pkl” file to the app.